In [31]:
import pandahouse as ph

In [32]:
connection_clickhouse = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'database': 'default',
    'user': 'student',
    'password': 'dpo_python_2020'}

In [33]:
successful_students = '''
WITH
    (SELECT MAX(toStartOfMonth(timest)) FROM default.peas) AS current_month
SELECT
    COUNT(st_id) AS successful_studs_cur_month
FROM
    (
    SELECT 
        st_id,
        SUM(correct) AS succesful
    FROM default.peas
    WHERE
        toStartOfMonth(timest) = current_month
    GROUP BY
        st_id
    HAVING
        succesful >= 20
    )
'''

In [34]:
ph.read_clickhouse(query=successful_students, connection=connection_clickhouse)

,successful_studs_cur_month
0,136


In [35]:
metrics = '''
SELECT
    test_grp AS group,
    ROUND(SUM(money) / COUNT(DISTINCT a.st_id)) AS ARPU,
    ROUND(sumIf(money, st_activity>10) / uniqIf(c.st_id, st_activity>10)) AS ARPAU,
    ROUND(uniqIf(b.st_id, money>0) / COUNT(DISTINCT a.st_id) * 100, 2) AS CR,
    ROUND(uniqIf(b.st_id, money>0 and st_activity>10) / uniqIf(c.st_id, st_activity>10) * 100, 2) AS CR_activity_to_purchase,
    ROUND(uniqIf(c.st_id, money>0 and st_math_activity>=2 and subject='Math') / uniqIf(c.st_id, st_math_activity>=2) * 100, 2) AS CR_math_activity_to_purchase
FROM default.studs AS a
LEFT JOIN 
    default.final_project_check AS b
ON 
    a.st_id=b.st_id
LEFT JOIN 
    (SELECT
        st_id,
        SUM(correct) AS st_activity,
        sumIf(correct, subject='Math') AS st_math_activity
    FROM
        default.peas
    GROUP BY st_id
    ) AS c
ON
    a.st_id=c.st_id
GROUP BY group
'''

In [36]:
ph.read_clickhouse(query=metrics, connection=connection_clickhouse)

,group,ARPU,ARPAU,CR,CR_activity_to_purchase,CR_math_activity_to_purchase
0,control,4541.0,10394.0,4.92,11.02,6.12
1,pilot,11508.0,29740.0,10.85,26.04,9.52
